3. Przygotowanie danych wejściowych do modelu i jego trening.

Przygotowane zgodnie ze skryptem Zapis_danych.ipynb pliki `signal.csv` i `bgd.csv` zawierają obliczone wartości zmiennych kinematycznych dla każdego zdarzenia. Każdy wiersz zawiera parametry opisujące jedno zdarzenie. Usuwamy kolumnę `Typ`, a następnie rozdzielamy dane na zbiór treningowy i walidacyjny. Sieć uczymy na zbiorze treningowym zawierającym po 50 tysięcy zdarzeń tła oraz sygnału, reszta danych przechodzi do zbioru walidacyjnego. Liczby zdarzeń w plikach "signal.csv" oraz "bgd.csv" są dobrane tak, aby w zbiorze walidacyjnym otrzymać odpowiedni stosunek tła do sygnału

In [ ]:

import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

# Wczytaj dane
signal_df = pd.read_csv("signal.csv").drop(columns=["Typ"])
background_df = pd.read_csv("bgd.csv").drop(columns=["Typ"])

X_signal = signal_df.to_numpy()
X_background = background_df.to_numpy()

# Podział na zbiór treningowy i walidacyjny. 
train_sig = X_signal[:50000]
val_sig   = X_signal[50000:]
train_bg  = X_background[:50000]
val_bg    = X_background[50000:]
# Tworzymy zbiór treningowy z przypadków sygnału oraz tła oraz wektory etykiet (klas) do treningu
X_train = np.concatenate([train_sig, train_bg])
y_train = np.concatenate([np.ones(len(train_sig)), np.zeros(len(train_bg))])
# Analogicznie tworzymy zbiór z danymi do testowania modelu
X_val = np.concatenate([val_sig, val_bg])
y_val = np.concatenate([np.ones(len(val_sig)), np.zeros(len(val_bg))])

# Konwertujemy wektory z etykietami do postaci wektorów z prawdopodobieństwem bycia daną klasą
y_train_cat = to_categorical(y_train, num_classes=2)
y_val_cat   = to_categorical(y_val, num_classes=2)

print("=== Rozkład danych ===")
print(f"Train sygnał: {np.sum(y_train==1)}")
print(f"Train tło:    {np.sum(y_train==0)}")
print(f"Val   sygnał: {np.sum(y_val==1)}")
print(f"Val   tło:    {np.sum(y_val==0)}")


=== Rozkład danych ===
Train sygnał: 50000
Train tło:    50000
Val   sygnał: 9685
Val   tło:    76092

Używamy modelu opisanego w pracy. Parametr TRESHOLD ustawia próg prawdopodobieńśtwa od którego przypadek uznawany jest za sygnał. Na jego postawie wypisywana jest liczba przewidywanych klas przez model. Trenujemy pięć modeli i uśredniamy ich predykcje aby uzsykać lepszą statystykę.

In [ ]:

from tensorflow.keras import layers, models
import tensorflow as tf

# Parametry modelu
N_MODELS = 5
THRESHOLD = 0.88
sizes = [512, 256, 128]
activation = "relu"

input_shape = X_train.shape[1]

def build_dnn_model(input_shape, sizes, act):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    for size in sizes:
        model.add(layers.Dense(size, activation=act))
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Trening kilku modeli i zapis predykcji walidacyjnych
val_predictions_list = []

for seed in range(N_MODELS):
    print(f"==> Trenuję model {seed+1}/{N_MODELS}...")
    tf.random.set_seed(seed)
    np.random.seed(seed)
    
    model = build_dnn_model(input_shape, sizes, activation)
    model.fit(X_train, y_train_cat, batch_size=50, epochs=10, validation_split=0.2, verbose=0)
    
    preds_val = model.predict(X_val, verbose=0)
    val_predictions_list.append(preds_val)

# Uśrednianie predykcji 
avg_val_preds = np.mean(val_predictions_list, axis=0)
predicted_classes = (avg_val_preds[:, 1] > THRESHOLD).astype(int)


Dla wytrenowanego modelu możemy wypisać jego predykcje oraz otrzymane wartości czystości i efektywności.

In [ ]:

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

y_true = np.argmax(y_val_cat, axis=1)
y_pred_classes = np.argmax(avg_val_preds, axis=1)

print("\n==> Metryki (średnia z predykcji wszystkich modeli):")
print(confusion_matrix(y_true, y_pred_classes))
print(classification_report(y_true, y_pred_classes, digits=4))

#  Statystyki predykcji 
unique, counts = np.unique(predicted_classes, return_counts=True)
results_dict = dict(zip(unique, counts))

print("\n==> Przewidywana liczba przypadków:")
print(f"  Tło (0): {results_dict.get(0, 0)}")
print(f"  Sygnał (1): {results_dict.get(1, 0)}")


==> Metryki (średnia z predykcji wszystkich modeli):
```
[[58858 17234]
 [  882  8803]]
              precision    recall  f1-score   support

           0     0.9852    0.7735    0.8666     76092
           1     0.3381    0.9089    0.4929      9685

    accuracy                         0.7888     85777
   macro avg     0.6617    0.8412    0.6797     85777
weighted avg     0.9122    0.7888    0.8244     85777


Przewidywana liczba przypadków:
  Tło (0): 76175
  Sygnał (1): 9602
```


  Następnie możemy narysować macierze klasyfikacji modelu oraz wykres czystości od efektywności.

In [ ]:
import seaborn as sns
# Macierz klasyfikacji
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Tło (0)", "Sygnał (1)"],
            yticklabels=["Tło (0)", "Sygnał (1)"])
plt.xlabel("Przewidywana klasa")
plt.ylabel("Rzeczywista klasa")
plt.title("Macierz konfuzji ")
plt.tight_layout()
plt.show()

#Rysujemy również macierz predykcji dla wybranej wartości pewności
chosen_threshold = 0.88 
y_pred_classes_treshold = (avg_val_preds[:, 1] > chosen_threshold).astype(int)
cm_tresh = confusion_matrix(y_true, y_pred_classes_treshold)

plt.figure(figsize=(5, 4))
sns.heatmap(cm_tresh, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Tło (0)", "Sygnał (1)"],
            yticklabels=["Tło (0)", "Sygnał (1)"])
plt.xlabel("Przewidywana klasa")
plt.ylabel("Rzeczywista klasa")
plt.title(f"Macierz konfuzji dla pewności: {chosen_threshold}")
plt.tight_layout()
plt.show()

from sklearn.metrics import precision_recall_curve

probs = avg_val_preds[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_true, probs)

plt.figure(figsize=(6,5))
plt.plot(recalls, precisions, color="red", label="Model")
plt.xlabel("Efektywność ")
plt.ylabel("Czystość ")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()





![Macierz klasyfikacji](images/Macierz_konfuzji_walidacja.png)
![Macierz klasyfikacji dla wybranej wartości pewności](images/Macierz_konfuzji_treshold.png)
![Krzywa czystość-efektywność](images/PR_curve.png)

Następnie możemy wyrysować jak wartości danego parametru wpływają na czystość oraz efektywność modelu.

In [ ]:

def histograms(parameter, inputs, targets, predictions, purity_filename, efficiency_filename):
    #Lista zmiennych
    parameter_dict = {
        'E_vis':    [0, 20, (1, 12), 'Energia widzialna [GeV]', 1], 
        'spheric':  [1, 20, (0, 1),   'Sferyczność', 1], 
        'pT':       [2, 20, (0, 1.5),  'Pęd poprzeczny [GeV]', 1], 
        'theta':    [3, 20, (0, 30), r'Kąt pędu całkowitego do osi $z$ [deg]', 1], 
        'pz':       [4, 20, (0, 10),'Pęd $p_z$ [GeV]', 1] 
    }

    idx, hist_bins, hist_range, x_label, multiplier = parameter_dict[parameter]
    # Przypisanie etykiet do zdarzeń
    true_labels = np.argmax(targets, axis=1)  
    # Podział na sygnał i tło
    signal_indices = np.where(true_labels == 1)[0]
    bg_indices = np.where(true_labels == 0)[0]
    # Dane do histogramów
    signal_hist_inputs = inputs[signal_indices, idx] * multiplier
    bg_hist_inputs = inputs[bg_indices, idx] * multiplier

    # True positives / false positives / false negatives
    true_positives = signal_hist_inputs[predictions[signal_indices] == 1]
    false_negatives = signal_hist_inputs[predictions[signal_indices] == 0]
    false_positives = bg_hist_inputs[predictions[bg_indices] == 1]

    # Histogramy liczności dla binów
    true_hist, edges = np.histogram(true_positives, bins=hist_bins, range=hist_range)
    false_pos_hist, _ = np.histogram(false_positives, bins=hist_bins, range=hist_range)
    false_neg_hist, _ = np.histogram(false_negatives, bins=hist_bins, range=hist_range)
    centers = (edges[:-1] + edges[1:]) / 2

    # Liczenie czystości
    purity = []
    purity_error = []
    for tp, fn in zip(true_hist, false_neg_hist):
        denom = tp + fp
        if denom > 0:
            p = tp / denom
            purity.append(p)
            purity_error.append(np.sqrt(tp * (1 - p)) / denom)
        else:
            purity.append(np.nan)
            purity_error.append(np.nan)

    #  Liczenie efektywności
    efficiency = []
    efficiency_error = []
    for tp, fp in zip(true_hist, false_pos_hist):
        denom = tp + fn
        if denom > 0:
            e = tp / denom
            efficiency.append(e)
            efficiency_error.append(np.sqrt(tp * (1 - e)) / denom)
        else:
            efficiency.append(np.nan)
            efficiency_error.append(np.nan)

    # Rysowanie histogramu z czystością
    fig, ax1 = plt.subplots()
    ax1.set_xlabel(x_label, fontsize=15)
    ax1.set_ylabel('N', color='tab:blue')
    ax1.hist(bg_hist_inputs, density=True, bins=50, range=hist_range, alpha=0.6, label='Tło')
    ax1.hist(signal_hist_inputs, density=True, bins=50, range=hist_range, alpha=0.6, label='Sygnał')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.legend()
    ax2 = ax1.twinx()
    ax2.set_ylabel('Czystość', color='tab:red', fontsize=15)
    ax2.errorbar(centers, purity, yerr=purity_error, linestyle='none', marker='o', color='tab:red', markersize=5)
    ax2.set_ylim([0, 1.1])
    fig.tight_layout()
    plt.savefig(purity_filename)
    plt.close(fig)

    # Rysowanie histogramu z efektywnością 
    fig, ax1 = plt.subplots()
    ax1.set_xlabel(x_label, fontsize=15)
    ax1.set_ylabel('N', color='tab:blue')
    ax1.hist(bg_hist_inputs, density=True, bins=50, range=hist_range, alpha=0.6, label='Tło')
    ax1.hist(signal_hist_inputs, density=True, bins=50, range=hist_range, alpha=0.6, label='Sygnał')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.legend()
    ax2 = ax1.twinx()
    ax2.set_ylabel('Efektywność', color='tab:red', fontsize=15)
    ax2.errorbar(centers, efficiency, yerr=efficiency_error, linestyle='none', marker='o', color='tab:red', markersize=5)
    ax2.set_ylim([0, 1.1])
    fig.tight_layout()
    plt.savefig(efficiency_filename)
    plt.close(fig)


# Wywołanie histogramów  
parameters = ['E_vis', 'pT', 'theta', 'pz', 'spheric']
y_pred_custom_threshold = (avg_val_preds[:, 1] > THRESHOLD).astype(int)

for param in parameters:
    purity_file = f"Czystosc_{param}.png"
    efficiency_file = f"Efektywnosc_{param}.png"
    histograms(param, X_val, y_val_cat, y_pred_custom_threshold, purity_file, efficiency_file)



Wykresy zapisują się do plików o nazwie "Czystosc/Efektywnosc_{nazwa parametru}.png". Poniżej pokazano przykładowy narysowany histogram.

![Czystosc_Evis.png](images/Czystosc_E_vis.png)